https://github.com/pola-rs/


https://www.ritchievink.com/blog/2021/02/28/i-wrote-one-of-the-fastest-dataframe-libraries/


https://www.kdnuggets.com/2020/12/rising-library-beating-pandas-performance.html


In [2]:
!pip install polars

  Using cached polars-0.15.14-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.8 MB)


In [3]:
import polars as pl
df = pl.DataFrame(
    {
        "A": [1, 2, 3, 4, 5],
        "fruits": ["banana", "banana", "apple", "apple", "banana"],
        "B": [5, 4, 3, 2, 1],
        "cars": ["beetle", "audi", "beetle", "beetle", "beetle"],
    }
)

In [5]:
df.head()

A,fruits,B,cars
i64,str,i64,str
1,"""banana""",5,"""beetle"""
2,"""banana""",4,"""audi"""
3,"""apple""",3,"""beetle"""
4,"""apple""",2,"""beetle"""
5,"""banana""",1,"""beetle"""


In [6]:
# embarrassingly parallel execution & very expressive query language
df.sort("fruits").select(
     [
         "fruits",
         "cars",
         pl.lit("fruits").alias("literal_string_fruits"),
         pl.col("B").filter(pl.col("cars") == "beetle").sum(),
         pl.col("A").filter(pl.col("B") > 2).sum().over("cars").alias("sum_A_by_cars"),
         pl.col("A").sum().over("fruits").alias("sum_A_by_fruits"),
         pl.col("A").reverse().over("fruits").alias("rev_A_by_fruits"),
         pl.col("A").sort_by("B").over("fruits").alias("sort_A_by_B_by_fruits"),
     ]
 )

fruits,cars,literal_string_fruits,B,sum_A_by_cars,sum_A_by_fruits,rev_A_by_fruits,sort_A_by_B_by_fruits
str,str,str,i64,i64,i64,i64,i64
"""apple""","""beetle""","""fruits""",11,4,7,4,4
"""apple""","""beetle""","""fruits""",11,4,7,3,3
"""banana""","""beetle""","""fruits""",11,4,8,5,5
"""banana""","""audi""","""fruits""",11,2,8,2,2
"""banana""","""beetle""","""fruits""",11,4,8,1,1
